# Retrieval Augmented Generation

LLMs excels at a wide range of tasks, but struggle with queries specific to your unique business context. This is where Retrieval Augmented Generation (RAG) becomes invaluable. RAG enables the LLM to leverage your internal knowledge bases or customer support documents, significantly enhancing its ability to answer domain-specific questions. Enterprises are increasingly building RAG applications to improve workflows in customer support, Q&A over internal company documents, financial & legal analysis, and much more.

In this guide, we'll demonstrate how to build and optimize a RAG system using the Anthropic documentation as our knowledge base. We'll walk you through:

1. Embeddings are from the `intfloat/multilingual-e5-large-instruct` model, where input is truncated to at most 512 tokens
2. In-memory vector database class is from Anthropic
3. Building a robust evaluation suite. We'll go beyond 'vibes' based evals and show you how to measure the retrieval pipeine & end to end performance independently
4. Implementing advanced techniques to improve RAG including summary indexing and re-ranking with Claude.

Through a series of targeted improvements, we achieved significant performance gains on the following metrics compared to a basic RAG pipeline (we'll explain what all these metrics *mean* in a bit)

## Table of Contents

1) Setup
2) Level 1 - Basic RAG
3) Building an Evaluation System

## Setup

We'll need a few libraries and models:

1. `intfloat/multilingual-e5-large-instruct` to generate high quality embeddings
2. `openai`,  LLM for (1) generation (2) judge
4. `pandas`, `numpy`, `matplotlib`, and `scikit-learn` for data manipulation and visualization


In [1]:
## silent setup (-q)
!pip install openai -q
!pip install pandas -q
!pip install numpy -q
!pip install matplotlib -q
!pip install seaborn -q
!pip install -U scikit-learn -q
!pip install sentence-transformers -q

In [2]:
import os
import getpass
from openai import OpenAI
OPENAI_API_KEY = getpass.getpass("Enter OpenAI API key")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# print(os.environ.get("OPENAI_API_KEY"))
client = OpenAI()

Enter OpenAI API key ········


### Downlaod the Embeddings model and run a quick test

In [3]:
from sentence_transformers import SentenceTransformer

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
# No need to add instruction for retrieval documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
]
input_texts = queries + documents

model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

embeddings = model.encode(input_texts, convert_to_tensor=True, normalize_embeddings=True)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())
# [[91.92853546142578, 67.5802993774414], [70.38143157958984, 92.13307189941406]]


/opt/vast-jupyter/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

[[91.92853546142578, 67.58030700683594], [70.38142395019531, 92.1330795288086]]


### Initialize a Vector DB Class

In this example, we're using an in-memory vector DB, but for a production application, you may want to use a hosted solution. 

In [4]:
import os
import pickle
import json
import numpy as np

class VectorDB:
    def __init__(self, name, api_key=None):
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/vector_db.pkl"

    def load_vec_db_in_memory(self, data):
        if self.embeddings and self.metadata:
            print("Vector database is already loaded. Skipping data loading.")
            return
        if os.path.exists(self.db_path):
            print("Loading vector database from disk.")
            self.load_vec_db()
            return

        texts = [f"Heading: {item['chunk_heading']}\n\n Chunk Text:{item['text']}" for item in data]
        self._embed_and_store(texts, data)
        self.save_db()
        print("Vector database loaded and saved.")

    def _embed_and_store(self, texts, data):
        batch_size = 128
        result = [
            model.encode(texts[i : i + batch_size])
            for i in range(0, len(texts), batch_size)
        ]
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = data

    def search(self, query, k=5, similarity_threshold=0.75):
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            query_embedding = model.encode(query)
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("No data loaded in the vector database.")

        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []
        
        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)
                
                if len(top_examples) >= k:
                    break
        # self.save_db()
        return top_examples

    def save_db(self):
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_vec_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Vector database file not found. Use load_vec_in_memory to create a new database.")
        with open(self.db_path, "rb") as file:
            data = pickle.load(file)
        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])

## Level 1 - Basic RAG

To get started, we'll set up a basic RAG pipeline using a bare bones approach. This is sometimes called 'Naive RAG' by many in the industry. A basic RAG pipeline includes the following 3 steps:

1) Chunk documents by heading - containing only the content from each subheading

2) Embed each document

3) Use Cosine similarity to retrieve documents in order to answer query

In [5]:
import json
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import logging
from typing import Callable, List, Dict, Any, Tuple, Set

def retrieve_similar(query, db):
    results = db.search(query, k=3)
    context = ""
    for result in results:
        chunk = result['metadata']
        context += f"\n{chunk['text']}\n"
    return results, context

def construct_prompt(query, context):
    # query = "How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool"
    
    prompt = f"""
    You have been tasked with helping us to answer the following query: 
    <query>
    {query}
    </query>
    You have access to the following documents which are meant to provide context as you answer the query:
    <documents>
    {context}
    </documents>
    Please remain faithful to the underlying context, and only deviate from it if you are 100% sure that you know the answer already. 
    Answer the question now, and avoid providing preamble such as 'Here is the answer', etc
    """

    return prompt

def answer_query_from_context(query, context):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": construct_prompt(query, context)
            }
        ],
        temperature=0.2
    )
    return completion.choices[0].message.content

logging.basicConfig(filename="log.log",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

# Load the evaluation dataset
with open('evaluation/docs_evaluation_dataset.json', 'r') as f:
    eval_data = json.load(f)

# Load the Anthropic documentation
with open('data/anthropic_docs.json', 'r') as f:
    anthropic_docs = json.load(f)

# Initialize the VectorDB
db = VectorDB("anthropic_docs")
db.load_vec_db_in_memory(anthropic_docs)

# test
query = "How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?"
context = ""'Creating Test Cases\n\n\nWhen you first access the Evaluation screen, you’ll see a single row:\n\nTo add more test cases:\nClick the ‘Add Test Case’ button.\nFill in values for each variable in your prompt.\nRepeat to create multiple scenarios.\nHere’s an example of a populated Evaluation screen with several test cases:\n\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\n\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\n'
print(retrieve_similar(query, db))
print(answer_query_from_context(query, context))

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Vector database loaded and saved.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

([{'metadata': {'chunk_link': 'https://docs.anthropic.com/en/docs/test-and-evaluate/eval-tool#creating-test-cases', 'chunk_heading': 'Creating Test Cases', 'text': 'Creating Test Cases\n\n\nWhen you first access the Evaluation screen, you’ll see a single row:\n\nTo add more test cases:\nClick the ‘Add Test Case’ button.\nFill in values for each variable in your prompt.\nRepeat to create multiple scenarios.\nHere’s an example of a populated Evaluation screen with several test cases:\n\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\n\nIf you update your original prompt text, you can re-run the entire eval suite against the new prompt to see how changes affect performance across all test cases.\nIf you update your 

## Eval Setup

When evaluating RAG applications, it's critical to evaluate the performance of the retrieval system and end to end system separately.

We synthetically generated an evaluation dataset consisting of 100 samples which include the following:
- A question
- Chunks from our docs which are relevant to that question. This is what we expect our retrieval system to retrieve when the question is asked
- A correct answer to the question.

This is a relatively challenging dataset. Some of our questions require synthesis between more than one chunk in order to be answered correctly, so it's important that our system can load in more than one chunk at a time. You can inspect the dataset by opening `evaluation/docs_evaluation_dataset.json`

Run the next cell to see a preview of the dataset

In [6]:
#previewing our eval dataset
import json

def preview_json(file_path, num_items=4):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            
        if isinstance(data, list):
            preview_data = data[:num_items]
        elif isinstance(data, dict):
            preview_data = dict(list(data.items())[:num_items])
        else:
            print(f"Unexpected data type: {type(data)}. Cannot preview.")
            return
        
        print(f"Preview of the first {num_items} items from {file_path}:")
        print(json.dumps(preview_data, indent=2))
        print(f"\nTotal number of items: {len(data)}")
        
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {file_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

preview_json('evaluation/docs_evaluation_dataset.json')

Preview of the first 4 items from evaluation/docs_evaluation_dataset.json:
[
  {
    "id": "efc09699",
    "question": "How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?",
    "correct_chunks": [
      "https://docs.anthropic.com/en/docs/test-and-evaluate/eval-tool#creating-test-cases",
      "https://docs.anthropic.com/en/docs/build-with-claude/develop-tests#building-evals-and-test-cases"
    ],
    "correct_answer": "To create multiple test cases in the Anthropic Evaluation tool, click the 'Add Test Case' button, fill in values for each variable in your prompt, and repeat the process to create additional test case scenarios."
  },
  {
    "id": "1305ea00",
    "question": "What embeddings provider does Anthropic recommend for customized domain-specific models, and what capabilities does this provider offer?",
    "correct_chunks": [
      "https://docs.anthropic.com/en/docs/build-with-claude/embeddings#before-implementing-embeddings",
      "h

## Defining Our Metric Calculation Functions

In [9]:
def calculate_mrr(retrieved_links: List[str], correct_links: Set[str]) -> float:
    for i, link in enumerate(retrieved_links, 1):
        if link in correct_links:
            return 1 / i
    return 0

def evaluate_retrieval(retrieval_function: Callable, evaluation_data: List[Dict[str, Any]], db: Any) -> Tuple[float, float, float, float, List[float], List[float], List[float]]:
    precisions = []
    recalls = []
    mrrs = []
    
    for i, item in enumerate(tqdm(evaluation_data, desc="Evaluating Retrieval")):
        try:
            retrieved_chunks, _ = retrieval_function(item['question'], db)
            retrieved_links = [chunk['metadata'].get('chunk_link', chunk['metadata'].get('url', '')) for chunk in retrieved_chunks]
        except Exception as e:
            logging.error(f"Error in retrieval function: {e}")
            continue

        correct_links = set(item['correct_chunks'])
        
        true_positives = len(set(retrieved_links) & correct_links)
        precision = true_positives / len(retrieved_links) if retrieved_links else 0
        recall = true_positives / len(correct_links) if correct_links else 0
        mrr = calculate_mrr(retrieved_links, correct_links)
        
        precisions.append(precision)
        recalls.append(recall)
        mrrs.append(mrr)
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(evaluation_data)} items. Current Avg Precision: {sum(precisions) / len(precisions):.4f}, Avg Recall: {sum(recalls) / len(recalls):.4f}, Avg MRR: {sum(mrrs) / len(mrrs):.4f}")
    
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0
    f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    
    return avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs

def evaluate_end_to_end(answer_query_function, db, eval_data):
    correct_answers = 0
    results = []
    total_questions = len(eval_data)
    
    for i, item in enumerate(tqdm(eval_data, desc="Evaluating End-to-End")):
        query = item['question']
        correct_answer = item['correct_answer']
        generated_answer = answer_query_function(query, db)
        
        comparision_prompt = f"""
        You are an AI assistant tasked with evaluating the correctness of answers to questions about Anthropic's documentation.
        
        Question: {query}
        
        Correct Answer: {correct_answer}
        
        Generated Answer: {generated_answer}
        
        Is the Generated Answer correct based on the Correct Answer? You should pay attention to the substance of the answer, and ignore minute details that may differ. 
        
        Small differences or changes in wording don't matter. If the generated answer and correct answer are saying essentially the same thing then that generated answer should be marked correct. 
        
        However, if there is any critical piece of information which is missing from the generated answer in comparison to the correct answer, then we should mark this as incorrect. 
        
        Finally, if there are any direct contradictions between the correct answer and generated answer, we should deem the generated answer to be incorrect.
        
        Respond in the following XML format (don't prefix with xml):
        <evaluation>
        <content>
        <explanation>Your explanation here</explanation>
        <is_correct>true/false</is_correct>
        </content>
        </evaluation>
        """
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": comparision_prompt}
                ],
                temperature=0.2,
            )
            response_text = str(response.choices[0].message.content)
            print(f'Query:\n{query}')
            print(f'Correct answer:\n{correct_answer}')
            print(f'Generated anser:\n{generated_answer}')
            print(f'Response_text from judge LLM:\n{response_text}')
            
            evaluation = ET.fromstring(response_text)
            is_correct_value = evaluation.find(".//is_correct").text
            
            is_correct = is_correct_value == 'true'
            
            if is_correct:
                correct_answers += 1
            results.append(is_correct)
            
            logging.info(f"Question {i + 1}/{total_questions}: {query}")
            logging.info(f"Correct: {is_correct}")
            logging.info("---")
            
        except ET.ParseError as e:
            logging.error(f"XML parsing error: {e}")
            is_correct = 'true' in response_text.lower()
            results.append(is_correct)
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            results.append(False)
        
        if (i + 1) % 10 == 0:
            current_accuracy = correct_answers / (i + 1)
            print(f"Processed {i + 1}/{total_questions} questions. Current Accuracy: {current_accuracy:.4f}")
        # time.sleep(2)
    accuracy = correct_answers / total_questions
    return accuracy, results

## Evaluating Our Base Case

In [10]:
import pandas as pd

avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs = evaluate_retrieval(retrieve_similar, eval_data, db)
e2e_accuracy, e2e_results = evaluate_end_to_end(answer_query_from_context, db, eval_data)

# Create a DataFrame
df = pd.DataFrame({
    'question': [item['question'] for item in eval_data],
    'retrieval_precision': precisions,
    'retrieval_recall': recalls,
    'retrieval_mrr': mrrs,
    'e2e_correct': e2e_results
})

# Save to CSV
from pathlib import Path
csv_dir = Path('evaluation/csvs')
csv_file_name = Path('evaluation_results_detailed.csv')
df.to_csv(csv_dir / csv_file_name, index=False)
print(f"Detailed results saved to {csv_dir/ csv_file_name}")

# Print the results
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average MRR: {avg_mrr:.4f}")
print(f"Average F1: {f1:.4f}")
print(f"End-to-End Accuracy: {e2e_accuracy:.4f}")

# Save the results to a file
json_dir = Path("evaluation/json_results")
result_file_name = Path("evaluation_results_one.json")
Path(json_dir).mkdir(parents=True, exist_ok=True)
with open(json_dir / result_file_name, 'w') as f:
    json.dump({
        "name": "Basic RAG",
        "average_precision": avg_precision,
        "average_recall": avg_recall,
        "average_f1": f1,
        "average_mrr": avg_mrr,
        "end_to_end_accuracy": e2e_accuracy
    }, f, indent=2)

print(f"Evaluation complete. Results saved to {json_dir / result_file_name}, {csv_dir/ csv_file_name}")

Evaluating Retrieval: 100%|██████████| 100/100 [00:00<00:00, 1781.85it/s]


Processed 10/100 items. Current Avg Precision: 0.4333, Avg Recall: 0.7000, Avg MRR: 0.9000
Processed 20/100 items. Current Avg Precision: 0.3333, Avg Recall: 0.5500, Avg MRR: 0.7000
Processed 30/100 items. Current Avg Precision: 0.3778, Avg Recall: 0.6000, Avg MRR: 0.7667
Processed 40/100 items. Current Avg Precision: 0.4083, Avg Recall: 0.6250, Avg MRR: 0.8000
Processed 50/100 items. Current Avg Precision: 0.4067, Avg Recall: 0.6300, Avg MRR: 0.7800
Processed 60/100 items. Current Avg Precision: 0.4056, Avg Recall: 0.6361, Avg MRR: 0.7833
Processed 70/100 items. Current Avg Precision: 0.3952, Avg Recall: 0.6167, Avg MRR: 0.7548
Processed 80/100 items. Current Avg Precision: 0.4208, Avg Recall: 0.6583, Avg MRR: 0.7792
Processed 90/100 items. Current Avg Precision: 0.4185, Avg Recall: 0.6556, Avg MRR: 0.7704
Processed 100/100 items. Current Avg Precision: 0.3933, Avg Recall: 0.6183, Avg MRR: 0.7333


Evaluating End-to-End:   1%|          | 1/100 [00:04<07:42,  4.68s/it]

Query:
How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?
Correct answer:
To create multiple test cases in the Anthropic Evaluation tool, click the 'Add Test Case' button, fill in values for each variable in your prompt, and repeat the process to create additional test case scenarios.
Generated anser:
To create multiple test cases for evaluation in the Anthropic Evaluation tool, you can follow these steps:

1. **Define Your Test Cases**: Start by outlining the different scenarios or inputs you want to evaluate. Each test case should have a specific goal or aspect of the model's performance that you want to assess.

2. **Format Your Inputs**: Ensure that each test case is formatted correctly according to the requirements of the Anthropic Evaluation tool. This may involve structuring your inputs in a specific way, such as using JSON or another data format.

3. **Use the Tool's Interface**: Access the Anthropic Evaluation tool and navigate to the se

Evaluating End-to-End:   2%|▏         | 2/100 [00:06<05:08,  3.15s/it]

...

Query:
What are the key differences between a pretrained language model like Claude's underlying model, and the final version of Claude available through Anthropic's API?
Correct answer:
The pretrained language model that forms Claude's foundation is not inherently good at answering questions or following instructions. To create the helpful, honest and safe Claude assistant available through the API, the pretrained model underwent fine-tuning and reinforcement learning from human feedback (RLHF).
Generated anser:
The key differences between a pretrained language model like Claude's underlying model and the final version of Claude available through Anthropic's API typically include the following aspects:

1. **Training Data and Fine-Tuning**: The pretrained model is trained on a broad dataset to learn general language patterns, while the final version of Claude is fine-tuned on specific datasets that may include additional curated data to enhance its performance on particular tasks or t

Evaluating End-to-End:  99%|█████████▉| 99/100 [06:02<00:04,  4.29s/it]

Query:
What is the IPv6 address range used by Anthropic?
Correct answer:
The IPv6 address range used by Anthropic is 2607:6bc0::/48.
Generated anser:
I currently do not have access to specific documents or databases to retrieve the IPv6 address range used by Anthropic. To find this information, you may need to consult official resources such as their website, network documentation, or contact their support team directly.
Response_text from judge LLM:
<evaluation>
<content>
<explanation>The generated answer does not provide the specific IPv6 address range used by Anthropic, which is a critical piece of information that is present in the correct answer. Instead, it suggests consulting other resources to find the information. Therefore, the generated answer is incorrect as it fails to convey the essential information regarding the IPv6 address range.</explanation>
<is_correct>false</is_correct>
</content>
</evaluation>


Evaluating End-to-End: 100%|██████████| 100/100 [06:05<00:00,  3.66s/it]

Query:
When using the Python SDK to create a message with Claude, what are two ways you can specify your API key?
Correct answer:
When using the Python SDK, you can specify your API key either by passing it as the api_key parameter when initializing the Anthropic client, or by setting it as an environment variable named ANTHROPIC_API_KEY which the client will use by default.
Generated anser:
You can specify your API key when using the Python SDK to create a message with Claude in two ways:

1. By setting the API key directly in your code, typically by assigning it to a variable or passing it as an argument when initializing the SDK client.
2. By using environment variables, where you can set the API key in your system's environment and the SDK will automatically read it from there. This is often done for security reasons to avoid hardcoding sensitive information in your code.
Response_text from judge LLM:
<evaluation>
<content>
<explanation>The generated answer correctly identifies two

In [11]:
!cat evaluation/json_results/evaluation_results_one.json 

{
  "name": "Basic RAG",
  "average_precision": 0.3933333333333335,
  "average_recall": 0.6183333333333334,
  "average_f1": 0.48081274025260856,
  "average_mrr": 0.7333333333333334,
  "end_to_end_accuracy": 0.27
}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
!cat evaluation/csvs/evaluation_results_detailed.csv

question,retrieval_precision,retrieval_recall,retrieval_mrr,e2e_correct
How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?,0.3333333333333333,0.5,1.0,False
"What embeddings provider does Anthropic recommend for customized domain-specific models, and what capabilities does this provider offer?",0.6666666666666666,1.0,1.0,False
"What are some key success metrics to consider when evaluating Claude's performance on a classification task, and how do they relate to choosing the right model to reduce latency?",0.6666666666666666,1.0,1.0,False
What are two ways that Claude for Sheets can improve prompt engineering workflows compared to using chained prompts?,0.3333333333333333,0.5,1.0,False
"What happens if a prompt for the Text Completions API is missing the ""\n\nHuman:"" and ""\n\nAssistant:"" turns?",0.6666666666666666,1.0,1.0,False
How do the additional tokens required for tool use in Claude API requests impact pricing compared to regular API reques

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
